# Importação de bibliotecas

In [1]:
import http.cookiejar
import urllib.request
import urllib.parse
import re
from collections import OrderedDict
from lxml.html import fragment_fromstring
from decimal import Decimal
import pandas as pd

# Coleta de dados

In [2]:
def todecimal(string):
  string = string.replace('.', '')
  string = string.replace(',', '.')

  if (string.endswith('%')):
    string = string[:-1]
    return Decimal(string) #/ 100
  else:
    return Decimal(string)

def get_data(*args, **kwargs):
    url = 'http://www.fundamentus.com.br/fii_resultado.php'
    cookie_jar = http.cookiejar.CookieJar()
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cookie_jar))
    opener.addheaders = [('User-agent', 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201'),
                         ('Accept', 'text/html, text/plain, text/css, text/sgml, */*;q=0.01')]

    data = {}

    with opener.open(url, urllib.parse.urlencode(data).encode('UTF-8')) as link:
        content = link.read().decode('ISO-8859-1')

    pattern = re.compile('<table id="tabelaResultado".*</table>', re.DOTALL)
    content = re.findall(pattern, content)[0]
    page = fragment_fromstring(content)
    result = OrderedDict()

    for rows in page.xpath('//tbody')[0].findall("tr"):
        result.update({rows.getchildren()[0][0].getchildren()[0].text: {
            'Segmento': rows.getchildren()[1].text,
            'Cotacao': todecimal(rows.getchildren()[2].text),
            'FFO Yield': todecimal(rows.getchildren()[3].text),
            'Dividend Yield': todecimal(rows.getchildren()[4].text),
            'P/VP': todecimal(rows.getchildren()[5].text),
            'Valor de Mercado': todecimal(rows.getchildren()[6].text),
            'Liquidez': todecimal(rows.getchildren()[7].text),
            'Qtd de imóveis': todecimal(rows.getchildren()[8].text),
            'Preço do m2': todecimal(rows.getchildren()[9].text),
            'Aluguel por m2': todecimal(rows.getchildren()[10].text),
            'Cap Rate': todecimal(rows.getchildren()[11].text),
            'Vacância Média': todecimal(rows.getchildren()[12].text)
        }})
    
    return result

In [3]:
data = get_data()
data

OrderedDict([('AAZQ11',
              {'Segmento': 'Títulos e Val. Mob.',
               'Cotacao': Decimal('7.39'),
               'FFO Yield': Decimal('20.37'),
               'Dividend Yield': Decimal('17.51'),
               'P/VP': Decimal('0.78'),
               'Valor de Mercado': Decimal('177636000'),
               'Liquidez': Decimal('1261870'),
               'Qtd de imóveis': Decimal('0'),
               'Preço do m2': Decimal('0.00'),
               'Aluguel por m2': Decimal('0.00'),
               'Cap Rate': Decimal('0.00'),
               'Vacância Média': Decimal('0.00')}),
             ('ABCP11',
              {'Segmento': 'Shoppings',
               'Cotacao': Decimal('67.50'),
               'FFO Yield': Decimal('10.17'),
               'Dividend Yield': Decimal('9.25'),
               'P/VP': Decimal('0.67'),
               'Valor de Mercado': Decimal('317863000'),
               'Liquidez': Decimal('57613'),
               'Qtd de imóveis': Decimal('1'),
         

In [4]:
papeis = {
    'Papel': [],
    'Segmento': [],
    'Cotacao': [],
    'FFO Yield': [],
    'Dividend Yield': [],
    'P/VP': [],
    'Valor de Mercado': [],
    'Liquidez': [],
    'Qtd de imóveis': [],
    'Preço do m2': [],
    'Aluguel por m2': [],
    'Cap Rate': [],
    'Vacância Média': []}

for papel, info in data.items():
    papeis['Papel'].append(papel)
    papeis['Segmento'].append(info['Segmento'])
    papeis['Cotacao'].append(info['Cotacao'])
    papeis['FFO Yield'].append(info['FFO Yield'])
    papeis['Dividend Yield'].append(info['Dividend Yield'])
    papeis['P/VP'].append(info['P/VP'])
    papeis['Valor de Mercado'].append(info['Valor de Mercado'])
    papeis['Liquidez'].append(info['Liquidez'])
    papeis['Qtd de imóveis'].append(info['Qtd de imóveis'])
    papeis['Preço do m2'].append(info['Preço do m2'])
    papeis['Aluguel por m2'].append(info['Aluguel por m2'])
    papeis['Cap Rate'].append(info['Cap Rate'])
    papeis['Vacância Média'].append(info['Vacância Média'])

df_fii = pd.DataFrame(papeis)
df_fii

,Papel,Segmento,Cotacao,FFO Yield,Dividend Yield,P/VP,Valor de Mercado,Liquidez,Qtd de imóveis,Preço do m2,Aluguel por m2,Cap Rate,Vacância Média
0,AAZQ11,Títulos e Val. Mob.,7.39,20.37,17.51,0.78,177636000,1261870,0,0.00,0.00,0.00,0.00
1,ABCP11,Shoppings,67.50,10.17,9.25,0.67,317863000,57613,1,4442.43,567.65,12.78,2.44
2,AEFI11,Outros,174.90,8.78,0.00,1.20,411893000,0,7,4312.36,390.35,9.05,0.00
3,AFCR11,None,103.15,7.43,0.00,1.03,394422000,0,0,0.00,0.00,0.00,0.00
4,AFHI11,Títulos e Val. Mob.,97.17,9.76,11.51,1.01,388384000,1484810,0,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,YUFI11,Residencial,89.84,3.84,3.59,0.85,44259700,4,40,2194.43,240.60,10.96,0.00
467,ZAGH11,Híbrido,12.00,41.06,1.24,0.12,5575150,465,0,0.00,0.00,0.00,0.00
468,ZAVC11,Títulos e Val. Mob.,10.01,0.16,3.63,1.01,31004500,116376,0,0.00,0.00,0.00,0.00
469,ZAVI11,Híbrido,103.74,13.02,14.00,0.78,121720000,422766,11,1541.49,185.28,12.02,0.63


# Tratamento dos dados

## Removendo valores nulos

In [5]:
df_fii.isnull().sum()

Papel               0
Segmento            4
Cotacao             0
FFO Yield           0
Dividend Yield      0
P/VP                0
Valor de Mercado    0
Liquidez            0
Qtd de imóveis      0
Preço do m2         0
Aluguel por m2      0
Cap Rate            0
Vacância Média      0
dtype: int64

In [6]:
df_fii.dropna(inplace=True)
df_fii

,Papel,Segmento,Cotacao,FFO Yield,Dividend Yield,P/VP,Valor de Mercado,Liquidez,Qtd de imóveis,Preço do m2,Aluguel por m2,Cap Rate,Vacância Média
0,AAZQ11,Títulos e Val. Mob.,7.39,20.37,17.51,0.78,177636000,1261870,0,0.00,0.00,0.00,0.00
1,ABCP11,Shoppings,67.50,10.17,9.25,0.67,317863000,57613,1,4442.43,567.65,12.78,2.44
2,AEFI11,Outros,174.90,8.78,0.00,1.20,411893000,0,7,4312.36,390.35,9.05,0.00
4,AFHI11,Títulos e Val. Mob.,97.17,9.76,11.51,1.01,388384000,1484810,0,0.00,0.00,0.00,0.00
5,AFOF11,Títulos e Val. Mob.,91.60,0.63,0.00,10.91,1390540000,0,0,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,YUFI11,Residencial,89.84,3.84,3.59,0.85,44259700,4,40,2194.43,240.60,10.96,0.00
467,ZAGH11,Híbrido,12.00,41.06,1.24,0.12,5575150,465,0,0.00,0.00,0.00,0.00
468,ZAVC11,Títulos e Val. Mob.,10.01,0.16,3.63,1.01,31004500,116376,0,0.00,0.00,0.00,0.00
469,ZAVI11,Híbrido,103.74,13.02,14.00,0.78,121720000,422766,11,1541.49,185.28,12.02,0.63


## Removendo valores com liquidez 0 pois são fundos interrompidos

In [7]:
df_fii['Liquidez'] = df_fii['Liquidez'].replace(0, '')
df_fii['Liquidez'] = pd.to_numeric(df_fii['Liquidez'], errors='coerce')
df_fii

,Papel,Segmento,Cotacao,FFO Yield,Dividend Yield,P/VP,Valor de Mercado,Liquidez,Qtd de imóveis,Preço do m2,Aluguel por m2,Cap Rate,Vacância Média
0,AAZQ11,Títulos e Val. Mob.,7.39,20.37,17.51,0.78,177636000,1261870.0,0,0.00,0.00,0.00,0.00
1,ABCP11,Shoppings,67.50,10.17,9.25,0.67,317863000,57613.0,1,4442.43,567.65,12.78,2.44
2,AEFI11,Outros,174.90,8.78,0.00,1.20,411893000,NaN,7,4312.36,390.35,9.05,0.00
4,AFHI11,Títulos e Val. Mob.,97.17,9.76,11.51,1.01,388384000,1484810.0,0,0.00,0.00,0.00,0.00
5,AFOF11,Títulos e Val. Mob.,91.60,0.63,0.00,10.91,1390540000,NaN,0,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,YUFI11,Residencial,89.84,3.84,3.59,0.85,44259700,4.0,40,2194.43,240.60,10.96,0.00
467,ZAGH11,Híbrido,12.00,41.06,1.24,0.12,5575150,465.0,0,0.00,0.00,0.00,0.00
468,ZAVC11,Títulos e Val. Mob.,10.01,0.16,3.63,1.01,31004500,116376.0,0,0.00,0.00,0.00,0.00
469,ZAVI11,Híbrido,103.74,13.02,14.00,0.78,121720000,422766.0,11,1541.49,185.28,12.02,0.63


In [8]:
df_fii.isnull().sum()

Papel                 0
Segmento              0
Cotacao               0
FFO Yield             0
Dividend Yield        0
P/VP                  0
Valor de Mercado      0
Liquidez            107
Qtd de imóveis        0
Preço do m2           0
Aluguel por m2        0
Cap Rate              0
Vacância Média        0
dtype: int64

In [9]:
df_fii.dropna(inplace=True)

## Convertendo valores para % nos campos aplicáveis

### Testando método de conversão

In [10]:
#df_fii['Dividend Yield'] = df_fii['Dividend Yield'].apply(lambda x: f'{x:.2f}%')
#df_fii['FFO Yield'] = df_fii['FFO Yield'].apply(lambda x: f'{x:.2f}%')
#df_fii['Cap Rate'] = df_fii['Cap Rate'].apply(lambda x: f'{x:.2f}%')
#df_fii['Vacância Média'] = df_fii['Vacância Média'].apply(lambda x: f'{x:.2f}%')

### Após método de conversão testado, transformação em função

In [11]:
def format_to_percent_column (df,col_name):
    df[col_name] = df[col_name].apply(lambda x: f'{x:.2f}%')

columns_format = ['Dividend Yield', 'FFO Yield','Cap Rate','Vacância Média']

for col in columns_format:
    format_to_percent_column(df_fii, col)

df_fii

,Papel,Segmento,Cotacao,FFO Yield,Dividend Yield,P/VP,Valor de Mercado,Liquidez,Qtd de imóveis,Preço do m2,Aluguel por m2,Cap Rate,Vacância Média
0,AAZQ11,Títulos e Val. Mob.,7.39,20.37%,17.51%,0.78,177636000,1261870.0,0,0.00,0.00,0.00%,0.00%
1,ABCP11,Shoppings,67.50,10.17%,9.25%,0.67,317863000,57613.0,1,4442.43,567.65,12.78%,2.44%
4,AFHI11,Títulos e Val. Mob.,97.17,9.76%,11.51%,1.01,388384000,1484810.0,0,0.00,0.00,0.00%,0.00%
7,AGRX11,Híbrido,9.71,13.84%,16.55%,0.96,173840000,295761.0,0,0.00,0.00,0.00%,0.00%
8,AIEC11,Lajes Corporativas,53.21,19.00%,17.66%,0.63,256738000,337791.0,2,10715.20,2098.55,19.58%,0.00%
...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,YUFI11,Residencial,89.84,3.84%,3.59%,0.85,44259700,4.0,40,2194.43,240.60,10.96%,0.00%
467,ZAGH11,Híbrido,12.00,41.06%,1.24%,0.12,5575150,465.0,0,0.00,0.00,0.00%,0.00%
468,ZAVC11,Títulos e Val. Mob.,10.01,0.16%,3.63%,1.01,31004500,116376.0,0,0.00,0.00,0.00%,0.00%
469,ZAVI11,Híbrido,103.74,13.02%,14.00%,0.78,121720000,422766.0,11,1541.49,185.28,12.02%,0.63%


# Exportação

In [12]:
df_fii.to_csv('data_fund\\fiis_fund.csv', index=False)
df_fii['Papel'].to_csv('data_fund\\fiis_tickers.csv', index=False)